Convert, inspect, flag and calibrate the exersize data set   
Understand and explain various choices and implementations

```
mvftoms.py ../../example-data/1554279147/1554279147/1554279147_sdp_l0.full.rdb -o 1554279147.ms -f -a -C '2150,3150' --quack=3 -r 'm001' --flags 'static'
```

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ls 1554279147.ms

In [ ]:
msfile='1554279147.ms'

In [ ]:
listobs(vis=msfile)

In [ ]:
flagdata(vis=msfile, mode='shadow');
flagdata(vis=msfile, mode='elevation', lowerlimit = 15);
flagdata(vis=msfile, mode='clip', clipminmax = [1e-5, 1000.0]);
flagdata(vis=msfile, mode='clip', field='', clipzeros=True, flagbackup=False);

In [ ]:
# antenna positions
plotants(vis=msfile)

In [ ]:
plotms(vis=msfile, xaxis='freq', yaxis='amp', coloraxis='corr')

In [ ]:
f_cal='PKS1934-63'
g_cal='PKS0408-65'

ref_ant = 'm001'

In [ ]:
plotms(vis=msfile,
       xaxis='freq',
       yaxis='phase',
       correlation='XX,YY',
       field=f_cal,
       coloraxis='baseline',
       plotrange=[0,0,-180,180])

In [ ]:
clearstat()
clearcal(msfile)

In [ ]:
plotms(vis=msfile,
       xaxis='time',
       yaxis='amp',
       correlation='XX,YY',
       field=f_cal,
       coloraxis='baseline')

In [ ]:
plotms(vis=msfile,
       xaxis='time',
       yaxis='phase',
       field=f_cal,
       coloraxis='baseline',
       plotrange=[0,0,-180,180])

In [ ]:
setjy(vis=msfile,
      field=f_cal,
      spw='0',
      scalebychan=True,
      standard='Perley-Butler 2010',
      fluxdensity=-1)

In [ ]:
gtable0 = msfile + '.G0'
rmtables(gtable0)

gaincal(vis=msfile,
        caltable=gtable0,
        field=f_cal,
        solint='inf',
        refant=ref_ant,
        spw='0:2150~3150',
        gaintype='G',
        calmode='p',
        minblperant=2,
        minsnr=3.0,
        gaintable=[''])

In [ ]:
plt.figure(figsize=[10,8])
plotcal(caltable=gtable0,
        xaxis='time',
        yaxis='phase',
        field=f_cal,
        spw = '0',
        plotrange = [-1, -1, -180, 180],
        fontsize=8,
        overplot=True,
        showgui=False)

In [ ]:
ktable = msfile + '.K'
rmtables(ktable)

#Solving for delays
gaincal(vis=msfile,
        caltable=ktable,
        field=f_cal,
        gaintype='K',
        solint='inf',
        refant=ref_ant,
        spw='0:2150~3150',
        combine='scan',
        minblperant=2,
        solnorm=False,
        minsnr=3.0,
        gaintable=[gtable0])

In [ ]:
plotcal(caltable = ktable, xaxis = 'antenna', yaxis = 'delay')

In [ ]:
btable = msfile + '.B'
rmtables(btable)

# Solve for the bandpass response -- complex gain as a function of channel across the passband.
bandpass(vis=msfile,
         caltable=btable,
         field=f_cal,
         refant=ref_ant,
         spw='0:2150~3150',
         solnorm=True,
         combine='scan',
         solint='inf',
         minblperant=2,
         bandtype='B',
         minsnr=3.0,
         gaintable=[gtable0, ktable])

In [ ]:
plotcal(caltable = btable, xaxis = 'chan', yaxis = 'phase')

In [ ]:
plotcal(caltable = btable, xaxis = 'chan', yaxis = 'amp')

In [ ]:
applycal(vis=msfile,
         gaintable=[btable, ktable],
         calwt=False,
         applymode='calflag')

In [ ]:
plotms(vis=msfile,
       xaxis='time',
       yaxis='amp',
       field=f_cal,
       coloraxis='baseline',
       plotrange=[0,0,-180,180])

In [ ]:
os.system('rm -rf '+msfile+'/table.lock');

In [ ]:
gtable = msfile + '.G'
rmtables(gtable)

gaincal(vis=msfile,
        caltable=gtable,
        field=f_cal,
        solint='int',
        refant=ref_ant,
        spw='0:2150~3150',
        combine='scan',
        gaintype='G',
        calmode='ap',
        minblperant=2,
        minsnr=1.0,
        solnorm=False,
        gaintable=[btable, ktable])

In [ ]:
plotcal(caltable = gtable, xaxis = 'time', yaxis = 'amp')

In [ ]:
plotcal(caltable = gtable, xaxis = 'time', yaxis = 'phase')

In [ ]:
applycal(vis=msfile,
         field=f_cal,
         gaintable=[ktable, btable, gtable],
         gainfield=[f_cal, f_cal, f_cal])

In [ ]:
plotms(vis=msfile,
       field=f_cal,
       xaxis='uvwave',
       yaxis='amp',
       correlation='XX,YY',
       ydatacolumn='corrected',
       coloraxis = 'corr')

In [ ]:
plotms(vis=msfile,
       field=f_cal,
       xaxis='uvwave',
       yaxis='phase',
       correlation='XX,YY',
       ydatacolumn='corrected',
       coloraxis = 'corr')

In [ ]:
split(vis=msfile, outputvis='PKS1934-63.ms', field=f_cal, datacolumn='corrected')

In [ ]:
clean(vis='PKS1934-63.ms',
      imagename = 'PKS1934-63',
      niter=20000,
      threshold='0.065Jy',
      psfmode='hogbom',
      imsize = 1024,
      cell='2.0arcsec',
      stokes = 'I',
      weighting='briggs',
      robust=0.0)

In [ ]:
viewer('PKS1934-63.image')

In [ ]:
gtable1 = msfile + '.G1'
rmtables(gtable1)

gaincal(vis=msfile,
        caltable=gtable1,
        field=g_cal,
        solint='int',
        refant=ref_ant,
        spw='0:2150~3150',
        combine='scan',
        gaintype='G',
        calmode='ap',
        minblperant=2,
        minsnr=1,
        solnorm=False,
        append=True,
        gaintable=[btable, ktable])

In [ ]:
pl.figure(figsize=[10,8])
plotcal(caltable=gtable1,
        xaxis='time',
        yaxis='phase',
        field=g_cal,
        spw='0',
        plotrange=[-1, -1, -180, 180],
        showgui=False)

In [ ]:
setjy(vis=msfile,
      field=f_cal,
      scalebychan=True,
      standard='Perley-Butler 2010',
      fluxdensity=-1)

In [ ]:
ftable = msfile + '.F'
rmtables(ftable)

fluxscale(vis=msfile,
          refspwmap=[0],
          caltable=gtable1,
          fluxtable=ftable,
          reference=[f_cal],
          transfer=[g_cal],
          append=False)

In [ ]:
applycal(vis=msfile,
         field=g_cal,
         gaintable=[ktable, btable, ftable],
         gainfield=[f_cal, f_cal, f_cal])

In [ ]:
split(vis=msfile, outputvis='PKS0408-65.ms', field=g_cal, datacolumn='corrected')

In [ ]:
clean(vis='PKS0408-65.ms',
      imagename = 'PKS0408-65',
      niter=20000,
      threshold='0.065Jy',
      psfmode='hogbom',
      imsize = 1024,
      cell='2.0arcsec',
      stokes = 'I',
      weighting='briggs',
      robust=0.0)

In [ ]:
viewer('PKS0408-65.image')